<a href="https://colab.research.google.com/github/zahrael97/Predicting-lung-cancer-survival-time/blob/master/Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder , OneHotEncoder

In [0]:
train_path = '/content/'

df_Radiomics_train = pd.read_csv(train_path + '/radiomics.csv',skiprows=[0,2], header=[0],index_col=0)

df_clinical_train = pd.read_csv(train_path + '/clinical_data.csv',index_col=0)

In [0]:
df_clinical_train = df_clinical_train.fillna(df_clinical_train.mean())
encoder = LabelEncoder()
encoder.fit(df_clinical_train["SourceDataset"] )
df_clinical_train["SourceDataset"]= encoder.transform(df_clinical_train["SourceDataset"])

In [0]:
df_X_train = pd.concat([df_Radiomics_train[["original_shape_Sphericity", "original_shape_SurfaceVolumeRatio", "original_shape_Maximum3DDiameter","original_firstorder_Entropy",
"original_glcm_Id","original_glcm_Idm"]],df_clinical_train[["SourceDataset","Nstage","age"]]], axis=1)

In [79]:
ytrain_path = '/content/'
df_y = pd.read_csv( ytrain_path+ '/output_VSVxRFU.csv', index_col=0)
p0 = df_y.loc[202]
print("p0.Event", p0.Event)
print("p0.SurvivalTime", p0.SurvivalTime)

p0.Event 0
p0.SurvivalTime 1378


In [0]:
test_path = '/content/sample_data/'
df_Radiomics_test = pd.read_csv(test_path +'radiomics.csv',skiprows=[0,2], header=[0],index_col=0)
df_clinical_test = pd.read_csv(test_path + 'clinical_data.csv',index_col=0)

In [0]:
df_clinical_test = df_clinical_test.fillna(df_clinical_test.mean())
encoder = LabelEncoder()
encoder.fit(df_clinical_test["SourceDataset"])
df_clinical_test["SourceDataset"] = encoder.transform(df_clinical_test["SourceDataset"])

In [0]:
df_X_test = pd.concat([df_Radiomics_test[["original_shape_Sphericity", "original_shape_SurfaceVolumeRatio", "original_shape_Maximum3DDiameter","original_firstorder_Entropy",
"original_glcm_Id","original_glcm_Idm"]],df_clinical_test[["SourceDataset","Nstage","age"]]], axis=1)

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper

import torch
import torchtuples as tt
import pycox
from pycox.datasets import metabric
from pycox.models import CoxPH
from pycox.evaluation import EvalSurv

In [84]:
pip install pycox

In [0]:
from sklearn.model_selection import train_test_split

y_train, y_test = train_test_split(df_y, test_size = 0.2, random_state=42)

In [0]:
df_train = pd.concat([df_X_train, y_train], axis = 1)

In [0]:
df_test = pd.concat([df_X_test, y_test], axis = 1)

In [0]:
np.random.seed(1234)
_ = torch.manual_seed(123)

In [89]:
df_train.head()

,original_shape_Sphericity,original_shape_SurfaceVolumeRatio,original_shape_Maximum3DDiameter,original_firstorder_Entropy,original_glcm_Id,original_glcm_Idm,SourceDataset,Nstage,age,SurvivalTime,Event
0,0.643333,0.438124,57.489129,5.187271,0.326798,0.244381,1.0,0.0,78.0000,NaN,NaN
1,0.551770,0.245677,80.012499,3.967976,0.559277,0.513317,0.0,2.0,71.7153,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,638.0,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,421.0,0.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,465.0,1.0


In [0]:

from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.semi_parametric import NonLinearCoxPHModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score

In [91]:
pip install pysurvival

In [92]:
df_train.head()

,original_shape_Sphericity,original_shape_SurfaceVolumeRatio,original_shape_Maximum3DDiameter,original_firstorder_Entropy,original_glcm_Id,original_glcm_Idm,SourceDataset,Nstage,age,SurvivalTime,Event
0,0.643333,0.438124,57.489129,5.187271,0.326798,0.244381,1.0,0.0,78.0000,NaN,NaN
1,0.551770,0.245677,80.012499,3.967976,0.559277,0.513317,0.0,2.0,71.7153,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,638.0,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,421.0,0.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,465.0,1.0


In [93]:
pip install keras==1.2.2

In [0]:
## Survival Analysis using Keras
import numpy as np

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.optimizers import SGD, RMSprop
from keras.regularizers import l2, activity_l2
import theano.tensor as T
import tensorflow as tf
import lifelines
from lifelines.utils import concordance_index
from lifelines import CoxPHFitter

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [95]:
pip install lifelines

In [0]:
#Loss Function
def negative_log_likelihood(E):
	def loss(y_true,y_pred):
		hazard_ratio = T.exp(y_pred)
		log_risk = T.log(T.extra_ops.cumsum(hazard_ratio))
		uncensored_likelihood = y_pred.T - log_risk
		censored_likelihood = uncensored_likelihood * E
		neg_likelihood = -T.sum(censored_likelihood)
		return neg_likelihood
	return loss


In [0]:
import pandas as pd
df = pd.concat([df_X_train, df_y], axis = 1)
E=df['SurvivalTime']
Y=df["Event"]
X=df
X=X.astype('float64')
X=X.iloc[:,2:]

X_train,X_val,Y_train,Y_val=train_test_split(X,Y,test_size=0.25, random_state=0)
X_train,X_val,E_train,E_val=train_test_split(X,E,test_size=0.25, random_state=0)

In [98]:
#Standardize
scaler=preprocessing.StandardScaler().fit(X_train.iloc[:,[1,6]])
X_train.iloc[:,[1,6]]=scaler.transform(X_train.iloc[:,[1,6]])
X_val.iloc[:,[1,6]]=scaler.transform(X_val.iloc[:,[1,6]])

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/py

In [0]:
#Sorting for NNL!
sort_idx = np.argsort(Y_train)[::-1]
X_train=X_train[sort_idx]
Y_train=Y_train[sort_idx]
E_train=E_train[sort_idx]


In [0]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [0]:
#from keras.regularizers import l2, activity_l2
#from keras.utils.visualize_util import to_graph
#from keras.models import Sequential
#import pydot
#to_graph(Sequential())
#Keras model
model = Sequential()
model.add(Dense(32, input_shape=(7,), init='glorot_uniform')) # shape= length, dimension
model.add(Activation('relu'))
model.add(Dense(32, init='glorot_uniform'))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation="linear", init='glorot_uniform', W_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
#

In [150]:

sgd = SGD(lr=1e-5, decay=0.01, momentum=0.9, nesterov=True)
rmsprop=RMSprop(lr=1e-5, rho=0.9, epsilon=1e-8)
model.compile(loss=negative_log_likelihood(E_train), optimizer=sgd)

NotImplementedError: ignored

In [126]:
print('Training...')
#model.fit(X_train, Y_train, batch_size=324, nb_epoch=1000, shuffle=False)  # Shuffle False --> Important!!


Training...


In [127]:
hr_pred=model.predict(X_train)
hr_pred=np.exp(hr_pred)
ci=concordance_index(Y_train,-hr_pred,E_train)

ValueError: ignored

In [128]:
hr_pred2=model.predict(X_val)
hr_pred2=np.exp(hr_pred2)
ci2=concordance_index(Y_val,-hr_pred2,E_val)
print 'Concordance Index for training dataset:', ci
print 'Concordance Index for test dataset:', ci2


SyntaxError: ignored

In [129]:
#Cox Fitting
cf = CoxPHFitter()
cf.fit(pd.concat([df_X_train, df_y], axis = 1),duration_col = 'SurvivalTime', event_col='Event') 
cf.print_summary()

TypeError: ignored